In [2]:
from Swimmer import *
import numpy as np
import matplotlib.pyplot as plt
from numba import njit

## Initialization

In [3]:
T=1
R=1
a = 10*R
epsilons = np.linspace(0.1,5,50)*R
epsilon = 3

steps = 1000
nsteps = 1000
dt = T/steps

r0 = np.array([[0, 0, 0],
[0, 0, a],
[a*np.sqrt(8)/3, 0 , -a/3],
[-a*np.sqrt(8)/6, a*np.sqrt(24)/6 , -a/3],
[-a*np.sqrt(8)/6, -a*np.sqrt(24)/6, -a/3]])

## P(t1,t2,t3,t4) = t2*t3*t4/(t1*t2*t3 + t1*t2*t4 + t1*t3*t4 + t2*t3*t4)

In [4]:
allPossbileActions = getAllPossibleActions(a,epsilon, steps)

NameError: name 'getAllPossibleActions' is not defined

In [15]:
state = np.array([1,1,1,1])

In [16]:
actionsData = np.zeros((64,4,3))

for i in range(64):
    print(i)
    _,delta,Rotation = act(allPossbileActions[i])
    actionsData[i,0,:] = delta
    actionsData[i,1:4,:] = Rotation
    
np.save("actionDatasForE=3",actionsData)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [17]:
@njit
def getAction(state,ratio):
    #t=1 if state=0 and t = ratio if state = 1   ???
    t1,t2,t3,t4 = np.where(state==0,1,ratio)
    sum = t1*t2*t3 + t1*t2*t4 + t1*t3*t4 + t2*t3*t4
    #probs = np.array([t2*t3*t4/sum,t1*t3*t4/sum,t1*t2*t4/sum,t2*t3*t1/sum])
    randomNumber = np.random.uniform(0,1,1)
    if randomNumber <= t4*t2*t3/sum:
        changingJoint = 0
    elif(randomNumber <= (t4*t3*t1+t4*t2*t3)/sum):  
        changingJoint = 1
    elif randomNumber <= (t4*t2*t1+t4*t2*t3+t4*t3*t1)/sum:
        changingJoint = 2
    else:
        changingJoint = 3
    return changingJoint,getDeltat2(t1,t2,t3,t4)


@njit
def doStep(Data,state,ratio):
    action,deltat = getAction(state,ratio)
    new_state = state.copy()
    new_state[action] = 1 - new_state[action]
    delta = Data[fromBinary(state)*4 + action][0,:]  #???
    rotation = Data[fromBinary(state)*4 + action][1:,:]
    return new_state,delta,rotation,deltat


@njit
def getDeltat(t1,t2,t3,t4,tc):
    tav = 18.844
    return ((1-1/(tav*t1))*(1-1/(tav*t2))*(1-1/(tav*t3))*(1-1/(tav*t4))*tav/tc)/(((1/t1 + 1/t2 + 1/t3 + 1/t4)**2)*(1-1/(tav*tc)))

@njit
def getDeltat2(t1,t2,t3,t4):
    return 1/(1/t1 + 1/t2 + 1/t3 + 1/t4)


@njit
def getBinary(a):
    return np.array([a//8,(a%8)//4,(a%4)//2,a%2])
@njit 
def fromBinary(binary):
    x = 0
    for i in range(len(binary)):
        x += binary[::-1][i]*(2**i)
    return x

@njit 
def fromBinary2d(binaries):
    x = np.zeros(binaries.shape[0],dtype=np.int64)
    for i in range(binaries.shape[0]):
        x[i] = fromBinary(binaries[i])
    return x



def getAllPossibleActions(a,epsilon,steps):
    
    closing = a - np.arange(steps+1)*epsilon/steps
    closed = (a-epsilon)*np.ones(steps+1)
    opening = a - epsilon + np.arange(steps+1)*epsilon/steps
    opened  = a *np.ones(steps+1)
    
    possibleActions = np.zeros((64,4,steps+1))
    
    
    
    for i in range(16):
        for j in range(4):
            actionBinaryState = getBinary(i)
            # closed for 0 and opened for 1 
            ca = np.array([closed,closed,closed,closed])*actionBinaryState.reshape(4,1) + np.array([opened,opened,opened,opened])*(1-actionBinaryState).reshape(4,1)
            if actionBinaryState[j] == 0:
                ca[j,:] = opening
            else:
                ca[j,:] = closing
                
            possibleActions[4*i+j] = ca 
        
    return possibleActions



@njit 
def mul(a,b):
    return a @ b


@njit
def mul2(a,v):
    ans = np.zeros(3,dtype=np.float64)
    for i in range(3):
        ans[i] = a[i,0]*v[0] + a[i,1]*v[1] +a[i,2]*v[2]
    return ans


@njit
def perturbe(Data,perturbingSteps,ratio):
    time = 0
    pertrubingPose = np.array([0.,0.,0.])
    perturbingRotation = np.eye(3)
    state = np.ones(4,dtype=np.int64)
    pertrubingPoses = np.zeros((perturbingSteps,3))
    pertrubingTimes = np.zeros(perturbingSteps)
        
    for t in range(perturbingSteps):
        newState,delta,rotation,deltat = doStep(Data,state,ratio)
        state = newState
        pertrubingPose += perturbingRotation @ delta
        perturbingRotation = rotation @ perturbingRotation
        pertrubingPoses[t] = pertrubingPose
        time += deltat
        pertrubingTimes[t] = time
        
    return pertrubingPoses,pertrubingTimes




@njit
def getPerturbingDiffusionCoefForRatio(Data,ratio,iterations=10_000,perturbingSteps=10_000):
    iterations = 10_000
    perturbingSteps = 10_000
    meanFinalPoses = np.zeros((perturbingSteps))
    meanFinalTimes = np.zeros(perturbingSteps)
    for iteration in range(iterations):
        pertrubingPoses,pertrubingTimes = perturbe(Data,perturbingSteps,ratio)
        #print(np.sum(np.square(pertrubingPoses[:]),axis=1).shape,meanFinalPoses.shape)
        meanFinalPoses +=   np.sum(np.square(pertrubingPoses[:]),axis=1)
        meanFinalTimes += pertrubingTimes
        #print(iteration)
     
    return meanFinalTimes/iterations,meanFinalPoses/iterations

def getPerturbingDiffusionCoef(Data,ratios):
    diffs =np.zeros(ratios.shape)
    for i in range(ratios.shape[0]):
        print(i)
        meanFinalTimes,meanFinalPoses= getPerturbingDiffusionCoefForRatio(Data,ratios[i])
        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(meanFinalPoses,meanFinalTimes)
        diffs[i] =  slope 
    return diffs


In [20]:
perturbe(actionsData,1000,1)

(array([[-2.62869533e-01, -4.55303387e-01, -1.85876829e-01],
        [-2.64005474e-01, -4.57270894e-01,  3.71245186e-01],
        [ 2.57985131e-01, -4.60122996e-01,  1.91416324e-01],
        ...,
        [-5.35865983e-01, -1.91867677e-02, -2.48636287e-01],
        [-3.01820976e-04,  3.49704169e-02, -3.71338945e-01],
        [ 2.03628862e-01,  4.78913571e-01, -1.14227347e-01]]),
 array([  0.25,   0.5 ,   0.75,   1.  ,   1.25,   1.5 ,   1.75,   2.  ,
          2.25,   2.5 ,   2.75,   3.  ,   3.25,   3.5 ,   3.75,   4.  ,
          4.25,   4.5 ,   4.75,   5.  ,   5.25,   5.5 ,   5.75,   6.  ,
          6.25,   6.5 ,   6.75,   7.  ,   7.25,   7.5 ,   7.75,   8.  ,
          8.25,   8.5 ,   8.75,   9.  ,   9.25,   9.5 ,   9.75,  10.  ,
         10.25,  10.5 ,  10.75,  11.  ,  11.25,  11.5 ,  11.75,  12.  ,
         12.25,  12.5 ,  12.75,  13.  ,  13.25,  13.5 ,  13.75,  14.  ,
         14.25,  14.5 ,  14.75,  15.  ,  15.25,  15.5 ,  15.75,  16.  ,
         16.25,  16.5 ,  16.75,  17.  ,  17

In [24]:
getPerturbingDiffusionCoefForRatio(actionsData,1,1000,1000)

(array([2.50000e-01, 5.00000e-01, 7.50000e-01, ..., 2.49950e+03,
        2.49975e+03, 2.50000e+03]),
 array([0.31095176, 0.31208896, 0.31354931, ..., 0.67959994, 0.68417648,
        0.68323087]))

In [22]:
getPerturbingDiffusionCoefForRatio

CPUDispatcher(<function getPerturbingDiffusionCoefForRatio at 0x7f19bfbb7600>)

In [8]:
ratios = np.exp(np.linspace(-5,5,100))

In [9]:
diffs = getPerturbingDiffusionCoef(actionsData,ratios)

0
1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
plt.plot(np.log(ratios),diffs)

In [ ]:
2+2

4